In [1]:
import pandas as pd
import glob  
import sys
import os
import time
import re
import numpy as np
from tqdm import tqdm

# Get Data with filtering

Get keyword list

In [2]:
keyword_list = pd.read_excel("../data/Text_keyword_0620.xlsx")

In [3]:
print(len(keyword_list))
keyword_list.head()

65


,Unnamed: 0,Target 키워드
0,1,인구
1,2,도시
2,3,도심
3,4,지역
4,5,지자체


In [4]:
DATA_PATH = '../data/pkl/'


df_filter = pd.DataFrame()

df_len = []
city_list = []
total_before_len = 0

for f in tqdm(glob.glob(f'{DATA_PATH}*.pkl')):
    if '시' in f:
        city = re.search(f'{DATA_PATH}(.+?)_merged.pkl', f).group(1)
        df = pd.read_pickle(f)
        df['city'] = city
        city_list.append(city)
        city_before_len = len(df)
        total_before_len += city_before_len
        filter_keyword = []
        for idx, keyword in enumerate(df['키워드']):
            keyword =  str(keyword).split(',')
            filter_keyword.append(set(keyword).intersection(set(keyword_list['Target 키워드'])))
        df['filter_keyword'] = filter_keyword
        filtered_df = df.loc[df['filter_keyword'].map(len) != 0]
        #print(f"City = {city}, before len = {city_before_len}, after len = {len(filtered_df)}")
        df_filter = pd.concat([df_filter, filtered_df])        
        temp = [city,  city_before_len]
        df_len.append(temp)
pd.DataFrame(df_len, columns=['city', 'count']).to_csv("../result/before_filter_city_count.csv")

#print(f"before : {total_before_len}, after : {len(df_filter)}")

100%|██████████| 82/82 [06:03<00:00,  4.43s/it]


In [5]:
keyword_filter_count = pd.DataFrame(df_filter['city'].value_counts()).reset_index()
keyword_filter_count.columns = ['city', 'drop by keyword']
keyword_filter_count

,city,drop by keyword
0,광주시,364099
1,청주시,304501
2,창원시,205197
3,제주시,193851
4,전주시,192606
...,...,...
72,김천시,26270
73,영천시,23380
74,영주시,22609
75,문경시,21643


### 분석제외 여부에 따른 filtering

In [6]:
df = pd.read_pickle("../data/merge_every_article.pkl")

In [7]:
df['분석제외 여부'].value_counts()

예외        308915
중복        237502
중복, 예외     32316
Name: 분석제외 여부, dtype: int64

In [8]:
df.shape

(8376057, 20)

In [9]:
df_excep = df[df['분석제외 여부'].isnull()]

In [10]:
exception_filter_count = pd.DataFrame(df_excep['city'].value_counts()).reset_index()
exception_filter_count.columns = ['city', 'drop by except']
exception_filter_count

,city,drop by except
0,광주시,451980
1,청주시,396267
2,제주시,272249
3,창원시,259685
4,전주시,254850
...,...,...
72,김천시,31553
73,영천시,28229
74,영주시,27640
75,문경시,26327


### 위치 정보에 너무 많은 도시가 등장하는 도시들 파악

In [11]:
df.shape

(8376057, 20)

In [12]:
def find_general_news(article):
    related_city = []
    article = str(article).split(',')
    city = [x for i in article for x in city_list if i in x]
    related_city.append(city)
    return related_city


In [13]:
related_city = df['위치'].apply(lambda x : find_general_news(x))

In [14]:
df['related_city'] = related_city

In [15]:
idx_list = []
idx_general = []
for idx, city in enumerate(related_city):
    if len(city[0]) <5:
        idx_list.append(idx)
    else:
        idx_general.append(idx)

In [16]:
df_general = df.iloc[idx_list]

In [17]:
general_filter_count = pd.DataFrame(df_general['city'].value_counts()).reset_index()
general_filter_count.columns = ['city', 'drop by related city']
general_filter_count

,city,drop by related city
0,광주시,445607
1,청주시,408361
2,제주시,274700
3,창원시,263393
4,전주시,255850
...,...,...
72,동두천시,27440
73,영천시,25936
74,영주시,25488
75,문경시,24183


In [23]:
df_general_article = df.iloc[idx_general]
print(df_general_article.city.value_counts())
df_general_inPohang = df_general_article[df_general_article['city'] == '포항시']
#df.iloc[idx_general].to_excel("../data/5개이상 도시가 언급된 기사들.xlsx")

수원시    34847
고양시    30917
용인시    30323
성남시    29546
광주시    25877
       ...  
당진시     4344
문경시     4318
영천시     4304
계룡시     4180
영주시     4122
Name: city, Length: 77, dtype: int64


In [24]:
df_general_inPohang.to_excel("../data/5개이상 도시가 언급된 기사들_포항시.xlsx")

### 통합 분류1을 기준으로 필터링

In [25]:
df['통합 분류1'].value_counts()

지역>경기            662632
지역>강원            638138
지역>충남            581523
지역>경남            581476
경제>부동산           449638
                  ...  
스포츠>축구>국가대표팀        891
경제>외환               387
스포츠>야구>메이저리그        286
스포츠>야구>일본프로야구        56
006000000             1
Name: 통합 분류1, Length: 87, dtype: int64

In [21]:
'국제>예시' == re.compile("^국제")

False

In [26]:
import re

idx_lst = []
for idx, cat in enumerate(df['통합 분류1']):
    if re.compile('^국제').findall(cat):
        continue
    elif re.compile('^스포츠').findall(cat):
        continue
    elif re.compile('^정치>북한').findall(cat):
        continue
    elif re.compile('^정치>국회_정당').findall(cat):
        continue
    elif re.compile('^정치>선거').findall(cat):
        continue
    elif re.compile('^정치>외교').findall(cat):
        continue
    else:
        idx_lst.append(idx)

In [27]:
df_filter_cat = df.iloc[idx_lst]
df_filter_cat.reset_index(inplace = True, drop = True)

In [49]:
cat_filter_count = pd.DataFrame(df_filter_cat['city'].value_counts()).reset_index()
cat_filter_count.columns = ['city', 'drop by category']
cat_filter_count

,city,drop by category
0,광주시,441071
1,청주시,403616
2,제주시,268215
3,창원시,260467
4,전주시,253672
...,...,...
72,김천시,31296
73,영천시,28534
74,영주시,27602
75,문경시,26637


### Count # by city

In [44]:
df_len_original = pd.DataFrame(df_len)
df_len_original.columns = ['city', 'original length']

In [47]:
drop_count = pd.merge(pd.merge(pd.merge(pd.merge(df_len_original, keyword_filter_count), exception_filter_count), general_filter_count),cat_filter_count)

save

In [50]:
drop_count.to_excel("../result/220704_drop_count.xlsx")
print('done well')

done well
